In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, plot_confusion_matrix, confusion_matrix, plot_roc_curve, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
import xgboost as xgb

In [ ]:
df = pd.read_csv("../../Data/win_loss_completed_df.csv")

In [ ]:
df

In [ ]:
df = df.drop(columns=["team1","team2"])


In [ ]:
cols = list(df.drop(columns=["pts"]).columns)

In [ ]:
for col in cols:
    for row in df[cols].index:
        if df[col][row] == "Null":
            df[col][row] = np.nan

In [ ]:
exp_col = []
pos_col = []
for col in cols:
    if "exp" in col:
        exp_col.append(col)
    else:
        pos_col.append(col)

In [ ]:
for col in exp_col:
    for row in df[cols].index:
        if df[col][row] == "R":
            df[col][row] = 0

In [ ]:
hold_out_df = df[df["game_date"] == 2022]
working_df = df[df["game_date"] != 2022]
working_df = working_df.drop(columns=["game_date"])
hold_out_df = hold_out_df.drop(columns=["game_date"])

In [ ]:
X = working_df.drop(columns=["pts"])
y = working_df["pts"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2)

In [ ]:
for col in pos_col:
    X_train[col+" is null"] = df[col].isna()
    X_test[col+" is null"] = df[col].isna()


In [ ]:
num_imp = SimpleImputer(strategy='mean')
num_imp.fit(X_train)

X_train_imp = num_imp.transform(X_train)
X_test_imp = num_imp.transform(X_test)

In [ ]:
X_train_imp_df = pd.DataFrame(X_train_imp)
X_test_imp_df = pd.DataFrame(X_test_imp)

for col in X_train_imp_df.columns:
    X_train_imp_df[col] = X_train_imp_df[col].astype(float)
    X_test_imp_df[col] = X_test_imp_df[col].astype(float)

In [ ]:
tree = DecisionTreeRegressor()
tree.fit(X_train_imp_df, y_train)

y_hat_train = tree.predict(X_train_imp_df) 
y_hat_test = tree.predict(X_test_imp_df) 
print(mean_squared_error(y_train, y_hat_train))
print(mean_squared_error(y_test, y_hat_test))

In [ ]:
tree = DecisionTreeRegressor()
tree.fit(X_train_imp_df, y_train)

y_hat_train = tree.predict(X_train_imp_df) 
y_hat_test = tree.predict(X_test_imp_df) 
print(mean_squared_error(y_train, y_hat_train))
print(mean_squared_error(y_test, y_hat_test))

In [ ]:
catch = {}
w=.01
w_list = []
while w<=1:
    w_list.append(w)
    w+=.01
x_list = range(1,101)

for z in x_list:
    for y in w_list:
        for x in x_list:
            xgb_model = xgb.XGBRegressor(random_state=42, learning_rate=y, max_depth=x, n_estimators=z,)
            xgb_model.fit(X_train_imp_df, y_train)
            y_hat_train = xgb_model.predict(X_train_imp_df) 
            y_hat_test = xgb_model.predict(X_test_imp_df) 
#             catch["x"+"y"+"z"+"train"] = mean_squared_error(y_train, y_hat_train)
            catch[str(x)+"_"+str(y)+"_"+str(z)] = mean_squared_error(y_test, y_hat_test)
            
catch_df = pd.DataFrame(pd.DataFrame(catch, index=catch.values()).reset_index()["index"])

In [ ]:
catch_df